In [ ]:
import time
from sshtunnel import SSHTunnelForwarder
import paramiko
import socket

# Caminho do arquivo de chave privada (PPK convertido para PEM)
private_key_path = '/Users/familialima/Documents/GitHub/PROD/Projeto8020/AuxScripts/vpnkey/tsap-andersonrocha.pem'

# Configurações do SSH
hostname = 'gate.atsglobe.com'
port = 6500
username = 'tsap-andersonrocha'
remote_host = 'globe-prd-mysql-writer.atsglobe.local'
remote_port = 3306
local_port = 2222

# Configurações de reconexão
MAX_RETRIES = 5  # Máximo de tentativas de reconexão
RETRY_DELAY = 10  # Tempo de espera entre tentativas em segundos
KEEP_ALIVE_INTERVAL = 300  # Intervalo de pacotes keep-alive em segundos

def start_ssh_tunnel():
    """
    Estabelece o túnel SSH e mantém a conexão ativa.
    """
    try:
        print(f"Iniciando o túnel SSH para {hostname}...")
        tunnel = SSHTunnelForwarder(
            (hostname, port),
            ssh_username=username,
            ssh_private_key=private_key_path,
            remote_bind_address=(remote_host, remote_port),
            local_bind_address=('0.0.0.0', local_port)
        )
        tunnel.daemon_forward_servers = True
        tunnel.start()
        print(f"Túnel SSH ativo. Redirecionando localhost:{local_port} para {remote_host}:{remote_port}.")
        return tunnel
    except Exception as e:
        print(f"Erro ao iniciar o túnel SSH: {e}")
        return None

def monitor_tunnel():
    """
    Monitora e reconecta o túnel SSH caso ele seja encerrado.
    """
    tunnel = None
    retries = 0

    while True:
        if not tunnel or not tunnel.is_alive:
            print("Tentando estabelecer o túnel SSH...")
            if tunnel:
                tunnel.stop()  # Certifica-se de encerrar o túnel anterior antes de tentar novamente.

            tunnel = start_ssh_tunnel()

            if not tunnel:
                retries += 1
                if retries >= MAX_RETRIES:
                    print("Número máximo de tentativas alcançado. Saindo do monitoramento.")
                    break
                print(f"Falha ao estabelecer o túnel. Retentando em {RETRY_DELAY} segundos...")
                time.sleep(RETRY_DELAY)
                continue
            else:
                retries = 0  # Reseta o contador de falhas em caso de sucesso.

        try:
            while tunnel.is_alive:
                print(f"Túnel ativo. Monitorando...")
                time.sleep(KEEP_ALIVE_INTERVAL)
        except KeyboardInterrupt:
            print("Encerrando conexão manualmente...")
            break
        except Exception as e:
            print(f"Túnel SSH interrompido: {e}")
        finally:
            if tunnel:
                tunnel.stop()
                print("Túnel SSH encerrado. Tentando reconectar...")
            time.sleep(RETRY_DELAY)

if __name__ == "__main__":
    print("Iniciando monitoramento do túnel VPN...")
    monitor_tunnel()
